## Preprocessing (...again)

In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [54]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [55]:
# Is name really useless??
application_df['NAME'].value_counts().head(50)

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [56]:
# Checking other columns
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [57]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [58]:
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [59]:
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [60]:
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [61]:
# Based on the above counts, dropping STATUS and SPECIAL_CONSIDERATIONS columns seems to make sense
# as well as EIN
application_df.drop(columns=['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS'], inplace=True)

In [62]:
# Given there are many entries from several organizations and one might expect future applications from the same groups,
# it makes some sense to keep the name column and bin the most common organizations
names_to_replace = list(application_df['NAME'].value_counts()[application_df['NAME'].value_counts() < 10].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")
    
# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [63]:
# Affiliation seems to have two main categories and the rest are rare
# Let's bin them as well
affiliation_to_replace = ['Family/Parent', 'National', 'Regional']

# Replace in dataframe
for aff in affiliation_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [64]:
# Replace rare organizations
organization_to_replace = ['Co-operative', 'Corporation']

# Replace in dataframe
for org in organization_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [65]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [66]:
# Choose a cutoff value and create a list of application types to be replaced (same as before)
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [67]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [68]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [69]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encode = pd.get_dummies(application_df)
len(df_encode.columns)

260

In [72]:
# Split our preprocessed data into our features and target arrays
X = df_encode.drop('IS_SUCCESSFUL', axis=1)
y = df_encode['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [73]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [74]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 128)               33280     
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dense_19 (Dense)            (None, 1)                 33        
                                                                 
Total params: 43,649
Trainable params: 43,649
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 779us/step - loss: 0.4711 - accuracy: 0.7706
Epoch 2/100
804/804 [==============================] - 1s 782us/step - loss: 0.4449 - accuracy: 0.7835
Epoch 3/100
804/804 [==============================] - 1s 779us/step - loss: 0.4404 - accuracy: 0.7860
Epoch 4/100
804/804 [==============================] - 1s 785us/step - loss: 0.4395 - accuracy: 0.7869
Epoch 5/100
804/804 [==============================] - 1s 779us/step - loss: 0.4376 - accuracy: 0.7866
Epoch 6/100
804/804 [==============================] - 1s 787us/step - loss: 0.4368 - accuracy: 0.7879
Epoch 7/100
804/804 [==============================] - 1s 785us/step - loss: 0.4358 - accuracy: 0.7886
Epoch 8/100
804/804 [==============================] - 1s 787us/step - loss: 0.4351 - accuracy: 0.7884
Epoch 9/100
804/804 [==============================] - 1s 779us/step - loss: 0.4340 - accuracy: 0.7893
Epoch 10/100
804/804 [==============================] - 1s 782us/step - l

804/804 [==============================] - 1s 805us/step - loss: 0.4148 - accuracy: 0.7985
Epoch 80/100
804/804 [==============================] - 1s 804us/step - loss: 0.4154 - accuracy: 0.7986
Epoch 81/100
804/804 [==============================] - 1s 807us/step - loss: 0.4156 - accuracy: 0.7984
Epoch 82/100
804/804 [==============================] - 1s 807us/step - loss: 0.4157 - accuracy: 0.7980
Epoch 83/100
804/804 [==============================] - 1s 800us/step - loss: 0.4152 - accuracy: 0.7994
Epoch 84/100
804/804 [==============================] - 1s 790us/step - loss: 0.4162 - accuracy: 0.7981
Epoch 85/100
804/804 [==============================] - 1s 804us/step - loss: 0.4147 - accuracy: 0.7978
Epoch 86/100
804/804 [==============================] - 1s 789us/step - loss: 0.4146 - accuracy: 0.7987
Epoch 87/100
804/804 [==============================] - 1s 779us/step - loss: 0.4149 - accuracy: 0.7987
Epoch 88/100
804/804 [==============================] - 1s 789us/step - loss:

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5160 - accuracy: 0.7857 - 210ms/epoch - 785us/step
Loss: 0.5160185098648071, Accuracy: 0.7856559753417969


In [97]:
nn.save('AlphabetSoupCharity_Optimization.h5')

## Using kerastuner to search for optimal parameters

In [92]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32,
        max_value=256,
        step=8), activation=activation, input_dim=259))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=0,
            max_value=128,
            step=4),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [93]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [94]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 16s]
val_accuracy: 0.787405252456665

Best val_accuracy So Far: 0.787405252456665
Total elapsed time: 00h 07m 09s
INFO:tensorflow:Oracle triggered exit


In [95]:
top_hyper = tuner.get_best_hyperparameters(3)

In [96]:
for x in top_hyper:
    print(x.values)

{'activation': 'relu', 'first_units': 88, 'num_layers': 4, 'units_0': 29, 'units_1': 52, 'units_2': 24, 'units_3': 28, 'units_4': 108, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 128, 'num_layers': 1, 'units_0': 93, 'units_1': 28, 'units_2': 16, 'units_3': 52, 'units_4': 16, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 80, 'num_layers': 4, 'units_0': 17, 'units_1': 60, 'units_2': 12, 'units_3': 64, 'units_4': 92, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0042'}
